# Notebook 6: Business Applications
## Turning Model Insights into Marketing Decisions

**Learning Objectives:**
- Calculate marketing ROI by channel
- Optimize budget allocation
- Create what-if scenarios
- Generate actionable recommendations

---

## From Model to Business Value

The true value of MMM lies not in the model itself, but in the business decisions it enables.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

print("Ready for business applications")

## Marketing ROI Calculator

Calculate the return on investment for each marketing channel:

In [ ]:
def calculate_marketing_roi(channel_spend, channel_contribution):
    '''
    Calculate ROI for each marketing channel.

    ROI = (Incremental Revenue - Spend) / Spend
    '''

    roi_results = {}

    for channel in channel_spend.keys():
        spend = channel_spend[channel]
        revenue = channel_contribution[channel]

        if spend > 0:
            roi = (revenue - spend) / spend
            roi_results[channel] = {
                'spend': spend,
                'revenue': revenue,
                'profit': revenue - spend,
                'roi': roi * 100  # Convert to percentage
            }

    # Convert to DataFrame for easy viewing
    roi_df = pd.DataFrame(roi_results).T
    roi_df = roi_df.sort_values('roi', ascending=False)

    return roi_df

# Example ROI calculation
channel_spend = {
    'TV': 100000,
    'Digital': 80000,
    'SEM': 60000,
    'Sponsorship': 50000,
    'Content': 30000,
    'Radio': 20000
}

# Simulated contributions (would come from model)
channel_contribution = {
    'TV': 250000,
    'Digital': 180000,
    'SEM': 90000,
    'Sponsorship': 120000,
    'Content': 45000,
    'Radio': 25000
}

roi_df = calculate_marketing_roi(channel_spend, channel_contribution)
print("Marketing ROI by Channel:")
print(roi_df)

# Visualize ROI
plt.figure(figsize=(10, 6))
colors = ['green' if x > 100 else 'orange' if x > 50 else 'red'
          for x in roi_df['roi']]
plt.barh(roi_df.index, roi_df['roi'], color=colors)
plt.xlabel('ROI (%)')
plt.title('Marketing Channel ROI Analysis')
plt.axvline(x=100, color='black', linestyle='--', alpha=0.5, label='Break-even')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Budget Optimization

Find the optimal allocation of marketing budget across channels:

In [ ]:
def optimize_budget_allocation(total_budget, saturation_params):
    '''
    Optimize budget allocation across marketing channels.
    Uses saturation curves to find optimal spend levels.
    '''

    channels = list(saturation_params.keys())
    n_channels = len(channels)

    # Objective: Maximize total response
    def objective(allocation):
        total_response = 0
        for i, channel in enumerate(channels):
            spend = allocation[i]
            params = saturation_params[channel]
            # Beta-Gamma response
            response = params['alpha'] * (spend ** params['beta']) * np.exp(-params['gamma'] * spend)
            total_response += response
        return -total_response  # Negative for minimization

    # Constraint: Total budget
    def budget_constraint(allocation):
        return total_budget - sum(allocation)

    # Bounds: Non-negative spend
    bounds = [(0, total_budget * 0.5) for _ in range(n_channels)]

    # Initial guess: Equal allocation
    initial_guess = [total_budget / n_channels] * n_channels

    # Constraints
    constraints = {'type': 'eq', 'fun': budget_constraint}

    # Optimize
    result = minimize(objective, initial_guess,
                     method='SLSQP', bounds=bounds, constraints=constraints)

    # Create result dictionary
    optimal_allocation = {}
    for i, channel in enumerate(channels):
        optimal_allocation[channel] = result.x[i]

    return optimal_allocation

# Example optimization
saturation_params = {
    'TV': {'alpha': 150, 'beta': 0.8, 'gamma': 0.01},
    'Digital': {'alpha': 120, 'beta': 0.9, 'gamma': 0.02},
    'SEM': {'alpha': 80, 'beta': 0.7, 'gamma': 0.03},
    'Sponsorship': {'alpha': 100, 'beta': 0.6, 'gamma': 0.015}
}

total_budget = 300000
optimal = optimize_budget_allocation(total_budget, saturation_params)

# Visualize allocation
plt.figure(figsize=(10, 6))
channels = list(optimal.keys())
values = list(optimal.values())
plt.pie(values, labels=channels, autopct='%1.1f%%', startangle=90)
plt.title(f'Optimal Budget Allocation (Total: ${total_budget:,.0f})')
plt.show()

print("\nOptimal Allocation:")
for channel, amount in optimal.items():
    print(f"  {channel}: ${amount:,.0f} ({amount/total_budget*100:.1f}%)")

## What-If Scenario Analysis

Simulate the impact of different marketing strategies:

In [ ]:
def scenario_analysis(base_spend, scenarios):
    '''
    Analyze impact of different spending scenarios.
    '''

    results = []

    for scenario_name, spend_changes in scenarios.items():
        # Apply changes to base spend
        new_spend = base_spend.copy()
        for channel, change in spend_changes.items():
            new_spend[channel] = base_spend[channel] * (1 + change)

        # Calculate impact (simplified)
        total_spend = sum(new_spend.values())
        estimated_revenue = total_spend * 2.5  # Simplified ROI model

        results.append({
            'Scenario': scenario_name,
            'Total Spend': total_spend,
            'Est. Revenue': estimated_revenue,
            'Est. Profit': estimated_revenue - total_spend,
            'ROI %': ((estimated_revenue - total_spend) / total_spend) * 100
        })

    return pd.DataFrame(results)

# Define scenarios
base_spend = {'TV': 100000, 'Digital': 80000, 'SEM': 60000}

scenarios = {
    'Base': {},
    'Digital Focus': {'TV': -0.3, 'Digital': 0.5, 'SEM': 0.2},
    'Brand Building': {'TV': 0.5, 'Digital': -0.2, 'SEM': -0.2},
    'Performance Focus': {'TV': -0.5, 'Digital': 0.2, 'SEM': 0.5},
    'Reduced Budget': {'TV': -0.2, 'Digital': -0.2, 'SEM': -0.2}
}

scenario_results = scenario_analysis(base_spend, scenarios)
print("Scenario Analysis Results:")
print(scenario_results)

# Visualize scenarios
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Revenue vs Spend
axes[0].scatter(scenario_results['Total Spend'],
               scenario_results['Est. Revenue'], s=100)
for idx, row in scenario_results.iterrows():
    axes[0].annotate(row['Scenario'],
                    (row['Total Spend'], row['Est. Revenue']),
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
axes[0].set_xlabel('Total Spend ($)')
axes[0].set_ylabel('Estimated Revenue ($)')
axes[0].set_title('Revenue vs Spend by Scenario')
axes[0].grid(True, alpha=0.3)

# ROI comparison
axes[1].bar(scenario_results['Scenario'], scenario_results['ROI %'])
axes[1].set_xlabel('Scenario')
axes[1].set_ylabel('ROI (%)')
axes[1].set_title('ROI Comparison Across Scenarios')
axes[1].axhline(y=100, color='r', linestyle='--', alpha=0.5)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## Actionable Recommendations

Based on the model insights, generate specific recommendations:

In [ ]:
def generate_recommendations(roi_df, saturation_analysis):
    '''
    Generate actionable marketing recommendations.
    '''

    recommendations = []

    # ROI-based recommendations
    high_roi = roi_df[roi_df['roi'] > 150].index.tolist()
    if high_roi:
        recommendations.append({
            'Priority': 'High',
            'Action': 'Increase Investment',
            'Channels': high_roi,
            'Reason': 'ROI exceeds 150%, indicating high efficiency'
        })

    low_roi = roi_df[roi_df['roi'] < 50].index.tolist()
    if low_roi:
        recommendations.append({
            'Priority': 'High',
            'Action': 'Reduce or Optimize',
            'Channels': low_roi,
            'Reason': 'ROI below 50%, poor return on investment'
        })

    # Saturation-based recommendations
    for channel, status in saturation_analysis.items():
        if status == 'undersaturated':
            recommendations.append({
                'Priority': 'Medium',
                'Action': 'Test Increased Spend',
                'Channels': [channel],
                'Reason': 'Channel has not reached saturation point'
            })
        elif status == 'oversaturated':
            recommendations.append({
                'Priority': 'Medium',
                'Action': 'Reduce Spend',
                'Channels': [channel],
                'Reason': 'Diminishing returns, past optimal spend level'
            })

    return pd.DataFrame(recommendations)

# Example saturation analysis
saturation_analysis = {
    'TV': 'optimal',
    'Digital': 'undersaturated',
    'SEM': 'oversaturated',
    'Sponsorship': 'optimal',
    'Content': 'undersaturated',
    'Radio': 'oversaturated'
}

recommendations = generate_recommendations(roi_df, saturation_analysis)
print("\nMarketing Recommendations:")
print("="*60)
for _, rec in recommendations.iterrows():
    print(f"\nPriority: {rec['Priority']}")
    print(f"Action: {rec['Action']}")
    print(f"Channels: {', '.join(rec['Channels'])}")
    print(f"Reason: {rec['Reason']}")

## Summary and Next Steps

### What We've Accomplished:
Through this 6-notebook series, we've built a complete Marketing Mix Model using Neural Additive Models:

1. **Data Foundation**: Integrated multiple data sources with proper time alignment
2. **Feature Engineering**: Created 28+ Beta-Gamma features for marketing saturation
3. **Model Architecture**: Built interpretable NAM with business constraints
4. **Training**: Implemented proper time series validation
5. **Diagnostics**: Generated comprehensive performance visualizations
6. **Business Applications**: Translated model insights into actionable recommendations

### Key Business Outcomes:
- **ROI Calculation**: Measure effectiveness of each marketing channel
- **Budget Optimization**: Allocate resources for maximum impact
- **Scenario Planning**: Simulate different marketing strategies
- **Actionable Insights**: Specific recommendations for marketing teams

### The Power of NAM for MMM:
- **Interpretability**: Unlike black-box models, we can explain every prediction
- **Business Logic**: Monotonic constraints ensure sensible relationships
- **Saturation Modeling**: Captures diminishing returns in marketing
- **Flexibility**: Different architectures for different feature types

### Next Steps for Implementation:
1. **Pilot Testing**: Start with one product category or region
2. **A/B Testing**: Validate model recommendations with experiments
3. **Continuous Learning**: Update model with new data monthly
4. **Stakeholder Buy-in**: Share visualizations with marketing teams
5. **ROI Tracking**: Monitor actual vs predicted outcomes

### Remember:
The model is a tool for better decision-making, not a replacement for business judgment. Combine model insights with domain expertise for optimal results.

---

**Congratulations!** You now have the knowledge and tools to build production-ready Marketing Mix Models using Neural Additive Models. The combination of advanced machine learning with interpretability makes NAM ideal for marketing analytics.